In [2]:
import sys
import h5py
import pylab

import numpy as np
 
import plotly.plotly as py
import plotly.graph_objs as go

import scipy.stats as st      
from scipy import interpolate
from scipy.optimize import curve_fit
from scipy.misc import factorial

import matplotlib.pyplot as plt  

### Colors

In [3]:
purples=[[0.0,   '#fcf9f7'],
 [0.1666666666666666, '#edcfc9'],
 [0.3333333333333333, '#daa2ac'],
 [0.5,                '#bc7897'],
 [0.6666666666666666, '#925684'],
 [0.8333333333333333, '#5f3868'],
 [1.0,                '#2d1e3e']]

### Read EBSD .h5 file

In [10]:
EBSDfileName = 'Si-EBSDmaster-20kV.h5'

#open file to read only
EBSDfile = h5py.File(EBSDfileName, 'r')


#save the energy bin electron matrix
EBSD_energy_data = np.zeros([11,2001,2001])

#energy loss spectra
EBSD_Elost = np.zeros(11)

for i in range(0, 11):
    dataset =  EBSDfile['EMData']['EBSDmaster']['masterSPSH'][...][i,:,:]
    EBSD_energy_data[i,:,:] = dataset
    
    EBSD_Elost[i] = EBSD_energy_data[i,:,:].sum()
EBSDfile.close()    


### Read ECP .h5 file

In [1]:
ECPfileName = 'Si-ECPmaster-20kV.h5'

#open file to read only
ECPfile = h5py.File(ECPfileName, 'r')


#save the energy bin electron matrix
ECP_energy_data = np.zeros([1,2001,2001])

#energy loss spectra
ECP_Elost = np.zeros(16)

for i in range(0, 1):
    dataset =  ECPfile['EMData']['ECPmaster']['masterSPSH'][...][i,:,:]
    ECP_energy_data[i,:,:] = dataset
    
    ECP_Elost[15-i] = ECP_energy_data[i,:,:].sum()

ECPfile.close()  


NameError: name 'h5py' is not defined

### Read TKD .h5 files

In [12]:
TKD50fileName = 'Si-TKDmaster50-20kV.h5'

#open file to read only
TKD50file = h5py.File(TKD50fileName, 'r')


#save the energy bin electron matrix
TKD50_energy_data = np.zeros([16,2001,2001])

#energy loss spectra
TKD50_Elost = np.zeros(16)

for i in range(0, 16):
    dataset =  TKD50file['EMData']['TKDmaster']['masterSPSH'][...][i,:,:]
    TKD50_energy_data[i,:,:] = dataset

    TKD50_Elost[i] = TKD50_energy_data[i,:,:].sum()
TKD50file.close() 



[       0.               0.               0.               0.               0.
        0.               0.               0.               0.               0.
        0.               0.               0.               0.               0.
  5021926.3039988]
[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]


In [13]:
TKD250fileName = 'Si-TKDmaster250-20kV.h5'

#open file to read only
TKD250file = h5py.File(TKD250fileName, 'r')


#save the energy bin electron matrix
TKD250_energy_data = np.zeros([16,2001,2001])

#energy loss spectra
TKD250_Elost = np.zeros(16)

for i in range(0, 16):
    dataset =  TKD250file['EMData']['TKDmaster']['masterSPSH'][...][i,:,:]
    TKD250_energy_data[i,:,:] = dataset
    
    TKD250_Elost[i] = TKD250_energy_data[i,:,:].sum()

TKD250file.close() 



[  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   3.12787758e+00   5.95815633e+01
   2.80242582e+02   6.77025059e+02   1.47638422e+03   3.14181636e+03
   6.78813887e+03   1.99274997e+04   2.38454155e+06   1.66763398e+05]


In [14]:
#energy loss spectra with data from h5 file
traces = dict()

traces[str(0)] = go.Scatter(x=np.arange(11), y=ECP_Elost[::-1]/np.amax(ECP_Elost), name='ECP' )

traces[str(1)] = go.Scatter(x=np.arange(16), y=TKD50_Elost[::-1]/np.amax(TKD50_Elost), name='TKD 50' )
traces[str(2)] = go.Scatter(x=np.arange(16), y=TKD250_Elost[::-1]/np.amax(TKD250_Elost), name='TKD 250' )
traces[str(3)] = go.Scatter(x=np.arange(16), y=EBSD_Elost[::-1]/np.amax(EBSD_Elost), name='EBSD' )

data50 = [traces[str(3)]]
data = [traces[str(i)] for i in range(4)]

fig = go.Figure(data=data)
py.iplot(fig, filename='Si energy loss')

### Read TKD energy data from .txt file and bin data
#### Unlike the .h5 file which contains already binned values, we sampled here 10,000 last elastic scattering energies so we can have a closer look at their distribution.

In [19]:
energy_TKD50 = []

TKD50 = 'TKD_Si50_10000.txt'

with open(TKD50, "r") as dataTKD50:
    for line in dataTKD50: #read one line at a time
        if ('Total' not in line) and ('Number' not in line): 
            column = line.replace(':',' ').replace(',',' ').split()
            energy_TKD50.append(float(column[1]))

      
        
# the bins should be of integer width, because poisson is a discrete distribution
TKD50_lost_renorm = [(20. - e) for e in energy_TKD50] 
entries_50, bin_edges, patches = plt.hist(TKD50_lost_renorm, bins=11, range=[-0.5, 10.5])

print entries_50


[ 7742.  2246.     0.     0.     0.     0.     0.     0.     0.     0.
     0.]



In [20]:
energy_TKD250 = []

TKD250 = 'TKD_Si250_10000.txt'

with open(TKD250, "r") as dataTKD250:
    for line in dataTKD250: #read one line at a time
        if ('Total' not in line) and ('Number' not in line): 
            column = line.replace(':',' ').replace(',',' ').split()
            energy_TKD250.append(float(column[1]))

  
        
# the bins should be of integer width, because poisson is a discrete distribution
TKD250_lost_renorm = [(20. - e) for e in energy_TKD250] 
entries_250, bin_edges, patches = plt.hist(TKD250_lost_renorm, bins=11, range=[-0.5, 10.5])

print entries_250


[   31.  6732.  2229.   654.   280.    72.     0.     0.     0.     0.
     0.]



#### Energy ditribution for TKD will not look very smooth since the statistical distribution is limited by the thinnes of the samples. We decide here to fit the physically expected curve to this data.

In [21]:
# Poisson fit function, parameter lamb is the fit parameter
def poisson(k, lamb):
    return (lamb**k/factorial(k)) * np.exp(-lamb)


In [26]:
traces = dict()

# x_axis "bin" values
bin_middles = np.arange(0, 11, 1)
 

#EBSD

# fit Poisson funciton with curve_fit
parameters, cov_matrix = curve_fit(poisson, bin_middles, EBSD_Elost[::-1]/np.amax(EBSD_Elost)) 

# plot poisson-deviation with fitted parameter
x_plot = np.linspace(0, 6, 100)

y_new = poisson(x_plot, *parameters)
max_y_new =  np.amax(y_new)
y_norm = y_new/max_y_new

traces[str(0)] = go.Scatter(x=x_plot, y=y_norm, name='<b>EBSD<b>', fill='tozeroy', marker =dict(color='rgb(189,189,189)'), opacity=0.05)


#ECP - almost one energy

ecp_data = [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., ]
# fit with curve_fit

parameters, cov_matrix = curve_fit(poisson, bin_middles, ecp_data) 

# plot poisson-deviation with fitted parameter
x_plot = np.linspace(0, 6, 100)

y_new = poisson(x_plot, *parameters)
max_y_new =  np.amax(y_new)
y_norm = y_new/max_y_new

traces[str(3)] = go.Scatter(x=x_plot, y=y_norm, name='<b>ECP<b>',  fill='tozeroy',marker =dict(color='rgb(99,99,99)'), opacity=0.3 )

In [29]:
   
# fit with curve_fit
parameters, cov_matrix = curve_fit(poisson, bin_middles, entries_50/np.sum(entries_50)) 

# plot poisson-deviation with fitted parameter
x_plot = np.linspace(0, 6, 100)

y_new = poisson(x_plot, *parameters)
max_y_new =  np.amax(y_new)
y_norm = y_new/max_y_new


traces[str(2)] = go.Scatter(x=x_plot, y=y_norm, name='<b>TKD 50 nm<b>',
                                 line = dict(color='#5f3868',
                                            width = 2.2),
                                      opacity =  0.8
                    )

# fit with curve_fit
parameters, cov_matrix = curve_fit(poisson, bin_middles, entries_250/np.sum(entries_250)) 

# plot poisson-deviation with fitted parameter
x_plot = np.linspace(0, 6, 100)

y_new = poisson(x_plot, *parameters)
max_y_new =  np.amax(y_new)
y_norm = y_new/max_y_new


traces[str(1)] = go.Scatter(x=x_plot, y=y_norm, name='<b>TKD 250 nm<b>', 
                                line = dict(color='#daa2ac',
                                            width = 2.4),
                                      opacity =  0.8)






In [31]:

data = [traces[str(i)] for i in range(4)]


layout = go.Layout(
    autosize=False,
    width=700,
    height=400,

    xaxis=dict(
        titlefont=dict(
          family='Times New Roman',
          size=18
        ),
        range=[0, 4],
        title='<b>Energy lost spectrum (keV)<b>', 
        showgrid=False,
        showline=True,
        ticks='inside',
        ticktext=['<b>0<b>','<b>1<b>','<b>2<b>', '<b>3<b>', '<b>4<b>', '<b>5<b>', '<b>6<b>' ], 
        tickvals=[0, 1, 2, 3, 4, 5, 6],
        zeroline=False,
        mirror="ticks",
        tickfont=dict(
          family='Times New Roman',
          size=17
        ),
    ),
    yaxis=dict(
        titlefont=dict(
        family='Times New Roman',
        size=18
        ),
        range=[0, 1.2],
        mirror="ticks",
        showgrid=False,
        showline=True,
        title='<b>Normalised intensity (a. u.)<b>', 
        ticks='inside',
        #ticktext=['<b>0.25<b>','<b>0.5<b>', '<b>0.75<b>', '<b>1<b>' ], 
        tickvals=[1.5],
        tickfont=dict(
          family='Times New Roman',
          size=16
        ),
    ),
    legend=dict(
          x=9.01, y=0.9,
          font= dict(
              family='Times New Roman',
              size=18)
     ),
    showlegend=False,
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)'
    
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='Si Poisson TKD')
#py.image.save_as(fig, filename='Si_energyLossSpectra.pdf')

In [34]:
trace_EBSD = go.Heatmap(z=EBSD_energy_data[:,1100:1600,1100:1600].sum(axis=0), colorscale='RdBu')
trace_EBSD_g = go.Heatmap(z=EBSD_energy_data[:,1100:1600,1100:1600].sum(axis=0), colorscale='Greys')


data_EBSD=[trace_EBSD_g]

layout = go.Layout(
    autosize=False,
    width=500,
    height=500
)
fig_EBSD=go.Figure(data=data_EBSD, layout=layout)

py.iplot(fig_EBSD, filename='EBSD')   
#py.image.save_as(fig_EBSD, filename='TKDpaper/EBSDmap_g.pdf')

In [35]:
trace_ECP_rb = go.Heatmap(z=ECP_energy_data[:,1100:1600,1100:1600].sum(axis=0), colorscale='RdBu')
trace_ECP_g = go.Heatmap(z=ECP_energy_data[:,1100:1600,1100:1600].sum(axis=0), colorscale='Greys')


data_ECP=[trace_ECP_g]

layout = go.Layout(
    autosize=False,
    width=500,
    height=500
)
fig_ECP=go.Figure(data=data_ECP, layout=layout)

py.iplot(fig_ECP, filename='ECP')   
#py.image.save_as(fig_ECP, filename='TKDpaper/ECPmap_g.pdf')

In [36]:
trace_TKD50_rb = go.Heatmap(z=TKD50_energy_data[15:,1100:1600,1100:1600].sum(axis=0), colorscale=purples, reversescale=True, zauto=False, zmax=4.5)
trace_TKD50_g = go.Heatmap(z=TKD50_energy_data[15:,1100:1600,1100:1600].sum(axis=0), colorscale='Greys')


data_TKD50=[trace_TKD50_rb]

layout = go.Layout(
    autosize=False,
    width=500,
    height=500
)
fig_TKD50=go.Figure(data=data_TKD50, layout=layout)

py.iplot(fig_TKD50, filename='TKD50')   
#py.image.save_as(fig_TKD50, filename='TKDpaper/TKD50map_purples.pdf')

In [37]:
trace_TKD250_rb = go.Heatmap(z=TKD250_energy_data[12:,1100:1600,1100:1600].sum(axis=0), colorscale=purples, reversescale=True, zauto=False, zmax=4.5)
trace_TKD250_g = go.Heatmap(z=TKD250_energy_data[12:,1100:1600,1100:1600].sum(axis=0), colorscale='Greys')


data_TKD250=[trace_TKD250_rb]

layout = go.Layout(
    autosize=False,
    width=500,
    height=500
)
fig_TKD250=go.Figure(data=data_TKD250, layout=layout)

py.iplot(fig_TKD250, filename='TKD250')   
#py.image.save_as(fig_TKD250, filename='TKDpaper/TKD250map_purple.pdf')

In [38]:
#plot
tracesScan = dict()

xRange = 140
EBSD = EBSD_energy_data[0:,1420:1560,1350:1351].sum(axis=0)

tracesScan[str(0)] = go.Scatter(x=np.arange(xRange), y=(EBSD[:,0]-np.min(EBSD[:,0]))/np.amax(EBSD[:,0]-np.min(EBSD[:,0])),
                                                      name='EBSD', marker = dict(color = 'Black'),
                                opacity=0.8
)

ECP = ECP_energy_data[0:,1420:1560,1350:1351].sum(axis=0)
tracesScan[str(1)] = go.Scatter(x=np.arange(xRange), y=(ECP[:,0]-np.min(ECP[:,0]))/np.amax(ECP[:,0]-np.min(ECP[:,0]))+1.4, 
                                                   name='ECP', marker = dict(color = 'Black'),
                                                                             opacity=0.8
                               )                             
TKD50 = TKD50_energy_data[15:,1420:1560,1350:1351].sum(axis=0)
tracesScan[str(2)] = go.Scatter(x=np.arange(xRange), y=(TKD50[:,0]-np.min(TKD50[:,0]))/np.amax(TKD50[:,0]-np.min(TKD50[:,0]))+0.8,
                                                name='TKD50',marker = dict(color = '#5f3868'),  
                                                             opacity=0.8
                               )                              
                                

TKD250 = TKD250_energy_data[0:,1420:1560,1350:1351].sum(axis=0)
tracesScan[str(3)] = go.Scatter(x=np.arange(xRange), y=(TKD250[:,0]-np.min(TKD250[:,0]))/np.amax(TKD250[:,0]-np.min(TKD250[:,0]))+0.4,
                                                name='TKD250' , marker = dict(color = '#daa2ac' ), 
                                                                              opacity=1
                               )

dataScan = [tracesScan[str(i)] for i in range(4)]




layout = go.Layout(
    autosize=False,
    width=700,
    height=500,

    xaxis=dict(
        titlefont=dict(
          family='Times New Roman',
          size=18
        ),
        range=[0, 139],
        title='<b>Position (pixels)<b>', 
        #showgrid=False,
        showline=True,
        ticks='inside',
        ticktext=['<b>0<b>','<b>20<b>','<b>40<b>', '<b>60<b>', '<b>80<b>', '<b>100<b>', '<b>120<b>' ], 
        tickvals=[0, 20, 40, 60, 80, 100, 120],
        zeroline=False,
        mirror="ticks",
        tickfont=dict(
          family='Times New Roman',
          size=16
        ),
    ),
    yaxis=dict(
        titlefont=dict(
        family='Times New Roman',
        size=18
        ),
        range=[-0.2, 2.7],
        mirror="ticks",
        #showgrid=False,
        showline=True,
        title='<b>Normalised intensity (a. u.)<b>', 
        ticks='inside',
        #ticktext=['<b>0.25<b>','<b>0.5<b>', '<b>0.75<b>', '<b>1<b>' ], 
        tickvals=[3],
        tickfont=dict(
          family='Times New Roman',
          size=16
        ),
    ),
    legend=dict(
          x=9.01, y=0.9,
          font= dict(
              family='Times New Roman',
              size=14)
     ),
    showlegend=False,
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)'
    
)


figScan = go.Figure(data=dataScan, layout=layout)
py.iplot(figScan, filename='Siscan')
#py.image.save_as(figScan, filename='SiScan.pdf')